In [1]:
print("all_ok")

all_ok


In [2]:
from langchain_groq import ChatGroq

c:\LLMOPS\Project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
model=ChatGroq(model="qwen/qwen3-32b")

In [6]:
model.invoke("What is the capital of France?").content

'<think>\nOkay, so I need to figure out what the capital of France is. Let me start by recalling what I know about France. I remember that France is a country in Europe, and it\'s known for cities like Paris, Lyon, Marseille, and Bordeaux. But which one is the capital? I think Paris might be the capital. Wait, but how sure am I? Let me think.\n\nI know that Paris is a major city in France, famous for the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It\'s a big tourist destination. Also, I\'ve heard of Paris being referred to as the "City of Light." But does that mean it\'s the capital? Maybe. I also remember learning in school that the capital of France is Paris, but I need to confirm that. \n\nLet me consider other possibilities. Lyon is another large city; I think it was once the capital of the Roman Empire in the region, but that was a long time ago. I don\'t think it\'s the current capital. Marseille is a significant port city, but again, not the capital. Bordeaux is 

1.Data ingestion

In [8]:
from langchain_community.document_loaders import PyPDFLoader

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
import os

In [11]:
os.getcwd()

'c:\\LLMOPS\\Project\\notebook'

In [12]:
file_path=os.path.join(os.getcwd(),"data","sample.pdf")

In [13]:
loader=PyPDFLoader(file_path)

In [18]:
documents=loader.load()

In [19]:
documents[0].metadata

{'producer': 'PyPDF2',
 'creator': 'PyPDF',
 'creationdate': '',
 'subject': 'Neural Information Processing Systems http://nips.cc/',
 'publisher': 'Curran Associates, Inc.',
 'language': 'en-US',
 'created': '2017',
 'eventtype': 'Poster',
 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On English-to-

In [20]:
len(documents)

11

In [37]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=200,length_function=len)

In [51]:
docs=text_splitter.split_documents(documents)

In [52]:
len(docs)

42

In [53]:
docs[2].page_content

'efforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [31, 21, 13].\n∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started\nthe effort to evaluate this idea. Ashish, with Illia, designed and implemented the ﬁrst Transformer models and\nhas been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head\nattention and the parameter-free position representation and became the other person involved in nearly every\ndetail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and\ntensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and\nefﬁcient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and\nimplementing tensor2tensor, replacing our earlier codebase, greatly improving result

In [54]:
from langchain_community.vectorstores import FAISS

In [56]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [57]:
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 392.59it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
embedding_model.embed_query(docs[0].page_content)

In [60]:
len(embedding_model.embed_query(docs[0].page_content))

384

In [61]:
vector_store=FAISS.from_documents(docs,embedding_model)

In [66]:
relevant_docs=vector_store.similarity_search("what is self attention mechanism?",k=5)